In [4]:
import tensorflow as tf
import numpy as np
import tensorflow
from tensorflow.python.ops.variables import trainable_variables
from tensorflow.keras.layers import MaxPooling2D

#import data
(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.mnist.load_data()

x_train = np.array(x_train,np.float32)
x_test = np.array(x_test,np.float32)

#dataset param
num_classes = 10
num_features = 784

#training param
learning_rate = 0.001 
training_step  =1000
batch_size = 32
display_step = 100


num_inputs =28
timesteps=28
num_units = 32

#tf.data
data_train = tensorflow.data.Dataset.from_tensor_slices(
 (x_train, y_train)
)
data_train=  data_train.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [5]:
from tensorflow.keras import Model,layers
class LSTM(Model):
 def __init__(self):
  super(LSTM,self).__init__()
  self.lstm_layer=layers.LSTM(units=num_units)
  self.out = layers.Dense(num_classes)
 def call(self,x,is_training=False):
  x=self.lstm_layer(x)
  x=self.out(x)
  if not is_training:
   x= tf.nn.softmax(x)
  return x
lstm_net=LSTM()

In [10]:
#compute loss
def cross_entropy(x,y):
 y = tf.cast(y,tf.int64)
 loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=x)
 return tf.reduce_mean(loss)

#accuracy function
def accuracy(y_pred,y_true):
 correct_prediction = tf.equal(tf.argmax(y_pred,1),tf.cast(y_true,tf.int64))
 return tf.reduce_mean(tf.cast(correct_prediction,tf.float32),axis=-1)


optimizer = tf.optimizers.Adam(learning_rate)

def run_optimization(x,y):
 with tf.GradientTape() as g:
  pred =lstm_net(x,is_training=True)
  loss = cross_entropy(pred,y)
  trainable_variables = lstm_net.trainable_variables
 gradients = g.gradient(loss,trainable_variables)
 optimizer.apply_gradients(zip(gradients,trainable_variables))
 
 
for step, (batch_x,batch_y) in enumerate(data_train.take(training_step),1):
  run_optimization(batch_x,batch_y)
  if step % display_step ==0:
   pred = lstm_net(batch_x)
   loss = cross_entropy(pred,batch_y)
   acc = accuracy(pred,batch_y)
   print(f"step{step} loss:{loss} acc:{acc}")


2023-05-11 15:56:58.497167: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [60000,28,28]
	 [[{{node Placeholder/_0}}]]
2023-05-11 15:56:58.497503: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [60000,28,28]
	 [[{{node Placeholder/_0}}]]
2023-05-11 15:57:02.654580: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-11 15:57:03.037521: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.

step100 loss:2.2381744384765625 acc:0.40625
step200 loss:2.1200454235076904 acc:0.5625
step300 loss:2.0533645153045654 acc:0.5625
step400 loss:1.9543893337249756 acc:0.71875
step500 loss:1.953502893447876 acc:0.625
step600 loss:1.9263888597488403 acc:0.6875
step700 loss:1.9477057456970215 acc:0.71875
step800 loss:1.8767294883728027 acc:0.65625
step900 loss:1.837165117263794 acc:0.8125
step1000 loss:1.8022446632385254 acc:0.84375
